### 数据预处理 

In [4]:
import cv2
import glob
import time
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input, Flatten, Dropout, Dense
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from sklearn.utils import shuffle
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions

## 数据与处理selector

In [5]:
def get_train_data1(imgDir, size=224):
    imgDir = shuffle(imgDir)
    train_x = np.zeros((len(imgDir), size, size, 3), dtype=np.float32)
    train_y = np.zeros(len(imgDir), dtype=np.uint8)
    for index, file in enumerate(imgDir):        
        img = image.load_img(file, target_size=(size, size))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        train_x[index] = preprocess_input(x)    
        if 'dog' in file:
            train_y[index] = 1
    return train_x, train_y

In [8]:
# cat 0  dog 1
print('Proprecessing begin !')
t=time.time()
train_dir = glob.glob('train_all/*.jpg')
train_x, train_y = get_train_data1(train_dir)
t2 = time.time()
print(round(t2 - t, 2), 'seconds to preprocessing !')

Proprecessing begin !
209.35 seconds to preprocessing !


### ----------------------------------------------------------------------------------------------------------------------------------###

### 将预处理后的数据  0.5 Dropout

In [9]:
from keras.optimizers import SGD
from keras.applications.resnet50 import ResNet50
opt = SGD(lr=0.0001, momentum=0.9)
myinput = Input(shape=(224, 224, 3))
base_model = ResNet50(weights='imagenet', input_tensor=myinput, include_top=False)
x = Flatten()(base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)
# this is the model we will train
model = Model(myinput, predictions)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
best_model = ModelCheckpoint('model_best.h5', verbose=1, save_best_only=True)
model.fit(train_x, train_y, validation_split=0.2, shuffle=True, batch_size=16, epochs=10, callbacks=[best_model])
model.save('model_10.h5')

A local file was found, but it seems to be incomplete or outdated because the md5 file hash does not match the original value of a268eb855778b3df3c7506639542a6af so we will re-download the data.


Exception: URL fetch failure on https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5: None -- [Errno 60] Operation timed out

### 将预处理后的数据 0.75 Dropout

In [ ]:
from keras.optimizers import SGD
from keras.applications.resnet50 import ResNet50
opt = SGD(lr=0.0001, momentum=0.9)
myinput = Input(shape=(224, 224, 3))
base_model = ResNet50(weights='imagenet', input_tensor=myinput, include_top=False)
x = Flatten()(base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.75)(x)
predictions = Dense(1, activation='sigmoid')(x)
# this is the model we will train
model = Model(myinput, predictions)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
best_model = ModelCheckpoint('model_best.h5', verbose=1, save_best_only=True)
model.fit(train_x, train_y, validation_split=0.2, shuffle=True, batch_size=16, epochs=10, callbacks=[best_model])
model.save('model_10.h5')

### ------------------------------------------------------------------------------------------------------###
### 清理数据

In [15]:
import os
size = 224
model_test = load_model('model_best.h5')
cat_dir = glob.glob('cat/*.jpg')
for file_name in cat_dir:        
    img = image.load_img(file_name, target_size=(size, size))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    if model_test.predict(x) > 0.5:
        print(file_name)
        os.remove(file_name)

100%|██████████| 12500/12500 [01:31<00:00, 135.90it/s]


In [11]:
import os
size = 224
model_test = load_model('model_best.h5')
dog_dir = glob.glob('dog/*.jpg')
for file_name in dog_dir:        
    img = image.load_img(file_name, target_size=(size, size))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    if model_test.predict(x) < 0.5:
        print(file_name)
        os.remove(file_name)

### ------------------------------------------------------------------------------------------------------------------####
### 75% 的Dropout 重新训练

In [ ]:
# cat 0  dog 1
print('Proprecessing begin !')
t=time.time()
train_dir = glob.glob('clean_train/*.jpg')
train_x, train_y = get_train_data1(train_dir)
t2 = time.time()
print(round(t2 - t, 2), 'seconds to preprocessing !')

In [ ]:
from keras.optimizers import SGD
from keras.applications.resnet50 import ResNet50
opt = SGD(lr=0.0001, momentum=0.9)
myinput = Input(shape=(224, 224, 3))
base_model = ResNet50(weights='imagenet', input_tensor=myinput, include_top=False)
x = Flatten()(base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.75)(x)
predictions = Dense(1, activation='sigmoid')(x)
# this is the model we will train
model = Model(myinput, predictions)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
best_model = ModelCheckpoint('model_best.h5', verbose=1, save_best_only=True)
model.fit(train_x, train_y, validation_split=0.2, shuffle=True, batch_size=16, epochs=10, callbacks=[best_model])
model.save('model_10.h5')

### ------------------------------------------------------------------------------------------------------ ###
### Drawing model


In [ ]:
from keras.models import load_model
model = load_model('model_best.h5')
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_drawing.png',show_shapes=True)

### ------------------------------------------------------------------------------------------------------ ###
### Testing ###

In [ ]:
import os
size = 224
test_dir = glob.glob('test1/*.jpg')
mytest = np.zeros((12500, size, size, 3), dtype=np.float32)
test_path = r'E:\final\test1'
for file_name in (test_dir):
    index = int(file_name[6:-4]) - 1
    img = image.load_img(file_name, target_size=(size, size))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    mytest[index] = preprocess_input(x)

### ------------------------------------------------------------------------------------------------------ ###
### 写入文件

In [ ]:
##predict
with open('file3.csv','w') as f:
    f.write('id,label\n')

model_test = load_model('model_best.h5')

with open('file3.csv','a') as f:
    for i in range(len(test_dir)):
        predict = model_test.predict(mytest[i:i+1])
        predict = predict[0][0]
        f.write('{},{}\n'.format(i+1,predict))